In [4]:
import copy

In [11]:
class FordFolkersonMethodSolver:
    def __init__(self, edges):
        self.main_graph = copy.deepcopy(edges)
        self.auxiliary_graph = copy.deepcopy(edges)
        self.create_auxuliary_graph()
        self.main_graph = copy.deepcopy(self.auxiliary_graph)
        self.add_current_flow_to_main_graph()
        print(f"{self.main_graph} self.main_graphf")
        print(f"{self.auxiliary_graph} self.auxiliary_graph")
        
    def add_current_flow_to_main_graph(self):
        for node, edges in self.main_graph.items():
            for edge_flow in edges.values():
                edge_flow.append(0)
                  
    def create_auxuliary_graph(self):
        for node, edges in self.main_graph.items():
            for edge in edges:
                if not self.main_graph.get(edge).get(node):
                    self.auxiliary_graph[edge][node] = [0]
                    
    def get_divergence(self):
        a_plus = sum([edge[1] for edge in self.main_graph.get('s').values()])
        a_minus = sum([edges.get('s')[1] if edges.get('s') else 0 for edges in self.main_graph.values()])
        return a_plus - a_minus
    
    def update_auxiliary_graph(self, path):
        for edge in path:
            self.auxiliary_graph[edge[0]][edge[1]] = [(self.main_graph.get(edge[0]).get(edge[1])[0] - 
                                                       self.main_graph.get(edge[0]).get(edge[1])[1] + 
                                                       self.main_graph.get(edge[1]).get(edge[0])[1])]
            self.auxiliary_graph[edge[1]][edge[0]] = [(self.main_graph.get(edge[1]).get(edge[0])[0] -
                                                       self.main_graph.get(edge[1]).get(edge[0])[1] + 
                                                       self.main_graph.get(edge[0]).get(edge[1])[1])]
        print(f"{self.auxiliary_graph} self.auxiliary_graph")
        
    def solve(self):
        path = self.bfs()
        while path:
            theta = min([self.auxiliary_graph.get(edge[0]).get(edge[1])[0] for edge in path]) # theta
            print(f"path {path}")
            print(f"min {theta}")
            for edge in path:
                flow_in_path, flow_not_in_path = self.get_flows(theta, edge, path)
                self.main_graph.get(edge[0]).get(edge[1])[1] = flow_in_path 
                self.main_graph.get(edge[1]).get(edge[0])[1] = flow_not_in_path 
            print(f"{self.main_graph} self.main_graphh")
            self.update_auxiliary_graph(path)
            path = self.bfs()
        return self.get_divergence()
    
    def get_flows(self, theta, edge, path):
        edge_in_path_flow = max([0, (self.main_graph.get(edge[0]).get(edge[1])[1] -
                                     self.main_graph.get(edge[0]).get(edge[1])[1] +
                                     theta - 0)])
        edge_not_in_path_flow = max([0, (self.main_graph.get(edge[0]).get(edge[1])[1] -
                                         self.main_graph.get(edge[0]).get(edge[1])[1] +
                                         0 - theta)])
        return edge_in_path_flow, edge_not_in_path_flow
    
    def bfs(self):
        Q = ['s']
        current_node = None
        l = {'s': None}
        while Q and current_node != 't':
            current_node = Q.pop(0)
            edges = self.auxiliary_graph.get(current_node)
            for edge, flows in edges.items():
                if edge not in l and flows[0]:
                    l[edge] = current_node
                    Q.append(edge)
        if current_node == 't':
            path = []
            while current_node != 's':
                prev_node = current_node
                current_node = l.get(current_node)
                path.append((current_node, prev_node))
            path.reverse()
            return path
        
        return None
                
        
                
                

In [14]:
Edges = {
    's': {'a': [3], 'b': [2]},
    'a': {'b': [2], 't': [1]},
    'b': {'t': [2]},
    't': {}
}

In [15]:
solver = FordFolkersonMethodSolver(Edges).solve()
solver

{'s': {'a': [3, 0], 'b': [2, 0]}, 'a': {'b': [2, 0], 't': [1, 0], 's': [0, 0]}, 'b': {'t': [2, 0], 's': [0, 0], 'a': [0, 0]}, 't': {'a': [0, 0], 'b': [0, 0]}} self.main_graphf
{'s': {'a': [3], 'b': [2]}, 'a': {'b': [2], 't': [1], 's': [0]}, 'b': {'t': [2], 's': [0], 'a': [0]}, 't': {'a': [0], 'b': [0]}} self.auxiliary_graph
path [('s', 'a'), ('a', 't')]
min 1
{'s': {'a': [3, 1], 'b': [2, 0]}, 'a': {'b': [2, 0], 't': [1, 1], 's': [0, 0]}, 'b': {'t': [2, 0], 's': [0, 0], 'a': [0, 0]}, 't': {'a': [0, 0], 'b': [0, 0]}} self.main_graphh
{'s': {'a': [2], 'b': [2]}, 'a': {'b': [2], 't': [0], 's': [1]}, 'b': {'t': [2], 's': [0], 'a': [0]}, 't': {'a': [1], 'b': [0]}} self.auxiliary_graph
path [('s', 'b'), ('b', 't')]
min 2
{'s': {'a': [3, 1], 'b': [2, 2]}, 'a': {'b': [2, 0], 't': [1, 1], 's': [0, 0]}, 'b': {'t': [2, 2], 's': [0, 0], 'a': [0, 0]}, 't': {'a': [0, 0], 'b': [0, 0]}} self.main_graphh
{'s': {'a': [2], 'b': [0]}, 'a': {'b': [2], 't': [0], 's': [1]}, 'b': {'t': [0], 's': [2], 'a': [0]}

3